# Home First Model Predictions

Changes in this NB:
- this notebook utilized the saved max home cih score from the trained models in lieu of creating a new zscore (so that it doesn't reset for each sample of data)

This notebook is to load the models trained in 6.3.2 NB and make historical predictions to MEDECON_PRD.HomeFirst_Model_Outputs

This sklearn model was created when it was realized the verticapy POC models for those who followed the verticapy POC SNF recommendations were readmitting at higher rates than those who did not.



# Imports

In [1]:
#Importing libraries, loading the data

import timeit
#start = timeit.default_timer()
import pickle
import pandas as pd
import statsmodels.formula.api as sm
from datetime import date
import numpy as np
import math
import random
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.datasets
from sklearn.dummy import DummyClassifier
import pandas
import numpy as np
import pdb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import pickle

from datetime import datetime

import treeinterpreter as ti

#set matplot lib style to 538
plt.style.use('fivethirtyeight')

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', 500)

#pandas profiling
from pandas_profiling import ProfileReport

In [2]:
#comment to hide graphics
%matplotlib inline


In [3]:
#Vertica Library
import vertica_python
from verticapy.utilities import *

In [4]:

# Compare Algorithms
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
#from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import time
# creating zscores
import scipy.stats as stats

In [6]:
#get time of beginning of code
startCodeTime = time.time()

# User Defined Functions

In [7]:
#define functions to explore a pandas dataframe 
def data_explore_func(dataframe):
    '''explore dataframe columns, dtypes, info, unique values, min, max, mean, and check for NaNs/duplicates'''
    print('List of Dataframe Columns\n\n',dataframe.columns)
    #print('\n\nData Types in this Dataframe\n\n',dataframe.dtypes)
    print('\n\nInfo of this Dataframe\n')
    print(dataframe.info())
    print('\n\nUnique Values (by column) in this Dataframe\n\n',dataframe.nunique().sort_values(ascending=False))
    print('\n\nUnique Values (by column) as % in this Dataframe\n\n',(dataframe.nunique()/len(dataframe)*100).sort_values(ascending=False))
    print('\n\nUnique Values for Every Column as (%)\n')
    for col in dataframe.columns:
        print(col)
        print(dataframe[col].value_counts()/len(dataframe[col])*100)
        print('\n')
    print('\n\nSummary Statistics of this Dataframe\n\n',dataframe.describe(include='all'))
    print('\n\nNull Values by Column as (%)')
    print((dataframe.isna().sum()/len(dataframe)*100).sort_values(ascending=False))
    print('\n\nDuplicates in this Dataframe\n',len(dataframe[dataframe.duplicated()]))
    print('Duplicates as a % of total Dataframe\n',len(dataframe[dataframe.duplicated()])/len(dataframe)*100)

# Data Initialization

### Vertica Connection

In [8]:
#Read in values
with open(r'C:\Users\JJFRASC\Documents\CCx Projects\Vertica_Extractions\vertica1.txt') as f:
#version on VM
#with open(r'C:\Users\Product_Reporting_Sv\Desktop\Python Projects\JJFRASC\Vertica_Extractions\vertica1.txt') as f:
    vertica1 = f.read()
    f.close()
    
with open(r'C:\Users\JJFRASC\Documents\CCx Projects\Vertica_Extractions\vertica2.txt') as f:
#version on VM
#with open(r'C:\Users\Product_Reporting_Sv\Desktop\Python Projects\JJFRASC\Vertica_Extractions\vertica2.txt') as f:
    vertica2 = f.read()
    f.close()

# Build vertica connection
vert_str = {
            'host': 'P1VERTICA', #production environment
            #'host': 'D1VERTICA', #developer environment
            'port': 5433,
            'user': vertica1, #enter your username
            'password': vertica2,  #enter your password
            'database': 'P1DA01', #production environment
            #'database': 'D1DA01', #developer environment
            'read_timeout': 600,
            'unicode_error': 'strict',
            'ssl': True
            }

# NEED TO LIMIT YEAR AND CLIENT?
YEAR & count 
- 2019	38933
- 2020	129875
- 2021	182618
- 2022	306096
- 2023	87221

Plan:
- 2019 + 2020
- 2021
- 2022 1/2 way
- 2022 2/2 way
- 2023 in final table?

# NOTE NEED TO REMOVE WHERE STAY_DISCHARGE_DATE IS NULL OR AFTER FIRST RUN OF OPS

In [9]:
#limit to clients of interest and patients currently in HOS or don't have epi_homefirst_recc 
data_sql = '''
SELECT * FROM DATALAB_PRD.HomeFirst_Model_Inputs_DEV
WHERE CLIENT IN ('WELLCARE', 'FL BLUE', 'HORIZON', 'OPTIMA', 'AMBETTER') 
AND (STAY_DISCHARGE_DATE IS NULL OR EPI_HOS_HOMEFIRST_RECOMMENDATION IS NULL)
;
'''

In [10]:
%%time
#Vertica connection to extract data from Vertica table
with vertica_python.connect(**vert_str) as connection:
    #create connection
    cur = connection.cursor('dict')
    
    #get input data for models
    cur.execute(data_sql)
    claims_data = pd.DataFrame(cur.fetchall())

Wall time: 4.16 s


# Explore Input Data - EDA

### Claims Data

#### (operations data but labeled as claims for lazy code updates)

In [11]:
#data_explore_func(claims_data)

In [12]:
claims_data.CLIENT.value_counts()

WELLCARE    1732
AMBETTER     709
FL BLUE      423
HORIZON      164
OPTIMA        68
Name: CLIENT, dtype: int64

In [13]:
claims_data.dtypes

MEMBER_AGE                          object
SCOPE_FLAG                           int64
EPI_ADMIT_TYPE                       int64
EPI_IS_READMIT_30_DAY                int64
EPI_IS_READMIT_90_DAY                int64
                                     ...  
STAY_DISCHARGE_DATE                 object
EPI_HOS_HOMEFIRST_RECOMMENDATION    object
Discharge_To                        object
CLIENT                              object
DATA_SOURCE                         object
Length: 289, dtype: object

In [14]:
#get all object columns
cols_to_float = claims_data.columns[claims_data.dtypes.eq('object')].values

In [15]:
#define columns that are truly string objects
true_objects = [ 'MEMBER_ID', 'EPI_ID', 'PAID_THRU_DATE',
       'STAY_ADMIT_DATE', 'STAY_DISCHARGE_DATE', 'Discharge_To', 'CLIENT',
       'DATA_SOURCE', 'EPI_HOS_HOMEFIRST_RECOMMENDATION']
#remove the true objects from the columns to convert to numeric
cols_to_float  = [e for e in cols_to_float if e not in true_objects]

In [16]:
%%time
#convert these colums to floats
claims_data[cols_to_float] = claims_data[cols_to_float].apply(pd.to_numeric)#, errors='coerce')

Wall time: 1.8 s


In [17]:
#check if there are any that need to be deleted
claims_data.select_dtypes(include='object').columns.values

array(['MEMBER_ID', 'EPI_ID', 'PAID_THRU_DATE', 'STAY_ADMIT_DATE',
       'STAY_DISCHARGE_DATE', 'EPI_HOS_HOMEFIRST_RECOMMENDATION',
       'Discharge_To', 'CLIENT', 'DATA_SOURCE'], dtype=object)

In [18]:
claims_data.describe()

MEMBER_AGE  SCOPE_FLAG  EPI_ADMIT_TYPE  EPI_IS_READMIT_30_DAY  \
count  3096.000000      3096.0          3096.0            3096.000000   
mean     62.660853         1.0             9.0               0.204134   
std      16.610699         0.0             0.0               0.403133   
min       0.000000         1.0             9.0               0.000000   
25%      54.000000         1.0             9.0               0.000000   
50%      66.000000         1.0             9.0               0.000000   
75%      74.000000         1.0             9.0               0.000000   
max     101.000000         1.0             9.0               1.000000   

       EPI_IS_READMIT_90_DAY  EPI_CONFIRMED_PCP_7_DAYS_POST_DC  \
count            3096.000000                            3096.0   
mean                0.332364                               0.0   
std                 0.471137                               0.0   
min                 0.000000                               0.0   
25%                 0.000000                               0.0   
50%                 0.000000                               0.0   
75%                 1.000000                               0.0   
max                 1.000000                               0.0   

       EPI_HOS_HAS_COVID  EPI_IS_READMIT_W_COVID_90  EPI_HAS_HHA_AUTH  \
count        3096.000000                3096.000000       3096.000000   
mean            0.001615                   0.000646          0.000646   
std             0.040161                   0.025412          0.025412   
min             0.000000                   0.000000          0.000000   
25%             0.000000                   0.000000          0.000000   
50%             0.000000                   0.000000          0.000000   
75%             0.000000                   0.000000          0.000000   
max             1.000000                   1.000000          1.000000   

       EPI_HHA_AUTH_CNT  ...  ICD_CAT_nan  DRG_nan  DRG_DESCRIPTION_nan  \
count       3096.000000  ...  3096.000000   3096.0               3096.0   
mean           5.772574  ...     0.221899      1.0                  1.0   
std            0.069235  ...     0.415591      0.0                  0.0   
min            2.000000  ...     0.000000      1.0                  1.0   
25%            5.774043  ...     0.000000      1.0                  1.0   
50%            5.774043  ...     0.000000      1.0                  1.0   
75%            5.774043  ...     0.000000      1.0                  1.0   
max            5.774043  ...     1.000000      1.0                  1.0   

       MDC_CATEGORY_nan  MCG_CATEGORY_nan  PRIOR_POS_21  PRIOR_POS_31  \
count            3096.0            3096.0   3096.000000   3096.000000   
mean                1.0               1.0      0.458010      0.060401   
std                 0.0               0.0      0.498314      0.238266   
min                 1.0               1.0      0.000000      0.000000   
25%                 1.0               1.0      0.000000      0.000000   
50%                 1.0               1.0      0.000000      0.000000   
75%                 1.0               1.0      1.000000      0.000000   
max                 1.0               1.0      1.000000      1.000000   

       PRIOR_POS_61  PRIOR_POS_LTAC  PRIOR_POS_nan  
count   3096.000000     3096.000000    3096.000000  
mean       0.011628        0.003876       0.466085  
std        0.107221        0.062147       0.498929  
min        0.000000        0.000000       0.000000  
25%        0.000000        0.000000       0.000000  
50%        0.000000        0.000000       0.000000  
75%        0.000000        0.000000       1.000000  
max        1.000000        1.000000       1.000000  

[8 rows x 280 columns]

In [19]:
len(claims_data)

3096

In [20]:
#get how many columns there are
len(claims_data.columns)

289

In [21]:
#see columns that are all null
claims_data.columns[claims_data.isna().all()].tolist()

['EPI_ID_DRG_WEIGHTS', 'DRG_WEIGHT', 'STAY_DISCHARGE_DATE']

In [22]:
#DROP ANY COLUMNS THAT ARE ALL NULL
claims_data.dropna(axis=1, how='all', inplace=True)
#see how many columns we have now
len(claims_data.columns)

286

In [23]:
#see columns that are all null
claims_data.columns[claims_data.isna().any()].tolist()

[]

In [24]:
#look at when member age is less than 0
#consider recalculating it with member dob and current date?
print('there are', len(claims_data[claims_data.MEMBER_AGE<0]), 'rows that have member_age <0')

there are 0 rows that have member_age <0


# Data manipulation

In [25]:
#create column that is binary 1 if discharged home, 0 if not
#claims_data['Discharge_Home'] = np.where(claims_data.Discharge_To=='Home Without Care', 1, 
 #        np.where(claims_data.Discharge_To=='Home With Care', 1,0))

In [26]:
#check to see if succesfull
#claims_data.groupby(['Discharge_To', 'Discharge_Home']).count()['Readmit_90']


# Model Pipelines    


In [27]:
#load models

#prod version
#with open(r'\\ccx.carecentrix.com\shares\CCX\HomeFirst\Discharge Options\Prod\1_Automated_Process\Data\Model\OPTIMIZE_WC_model_dictionary_12072022_WC3yr.pkl', 'rb') as file:
with open(r'C:\Users\JJFRASC\Documents\CCx Projects\Advanced Data Labs Projects\POC-RiskStrat-SNFLikely_toVertica\models\DEV_OPS_HomeFirst_model_dictionary2023-06-29.pkl', 'rb') as file:    
    
    models = pickle.load(file) 
    #get the four models and the X columns needed to predict the target variable
    Home_pianobar = models['Home']
    Home_cols = models['Home_cols']
    SNF_pianobar = models['SNF']
    SNF_cols = models['SNF_cols']
    Readmission = models['RA_model']
    RA_model_cols = models['RA_model_cols']
    SNF_Likely = models['Discharge_model']
    SNF_Likely_model_cols = models['Discharge_model_cols']
    cihFor_zscore = 0.15#0.2465 # #models['cihFor_zscore']
    #get the pca/scaler and other datas
    #pca = models['pca_demographics']
    #Scaler = models['Scaler']
    #icd10 = models['dxs']
    #risks = models['dx_risks']
    #svi = models['svi_map']


In [28]:
cihFor_zscore

0.15

## Create Recommendations

In [29]:
%%time
print('making predictions...') #print for visual control
#make predictions from loaded models
claims_data['SNF_Discharge_Likelihood'] = SNF_Likely.predict_proba(claims_data[SNF_Likely_model_cols])[:,1]
claims_data['Readmission_Likelihood'] = Readmission.predict_proba(claims_data[RA_model_cols])[:,1]
claims_data['pred_Home_RA'] = Home_pianobar.predict_proba(claims_data[Home_cols])[:,1]
claims_data['pred_SNF_RA'] = SNF_pianobar.predict_proba(claims_data[SNF_cols])[:,1]



print('generating recommendations...')#print for visual control
#create change if home column
claims_data['ChangeIfHome'] =  claims_data['pred_Home_RA'] - claims_data['pred_SNF_RA']

#create zscore based on CIH variable
#putting in null b/c using traing data - max cih from home recc as zscore 
claims_data['zscore'] = np.nan#stats.zscore(claims_data['ChangeIfHome'])
#use max cih from home recc in place of zscore
#otherwise this recommendation will reset based on only that runs' sample
claims_data['Recommendation'] = np.where(claims_data.sickle==1,'Path of Care Override - Sickle Cell',
                                             np.where(claims_data.ChangeIfHome>cihFor_zscore, 'SNF', 'Home'))


#create column, if zscore above 90% threshold, send SNF else home
#claims_data['Recommendation'] = np.where(claims_data.zscore>1.645, 'SNF', 'Home')
#except when sickle cell
#claims_data['Recommendation'] = np.where(claims_data.sickle==1,'Path of Care Override - Sickle Cell',
 #                                            np.where(claims_data.zscore>1.96, 'SNF', 'Home'))


#create numerical column for Recommendation
#claims_data['RecommendationBinary'] = np.where(claims_data['Recommendation']=='Home', 1,0)




making predictions...
generating recommendations...
Wall time: 705 ms


In [30]:
claims_data.describe()['ChangeIfHome']

count    3096.000000
mean       -0.071958
std         0.140829
min        -0.790000
25%        -0.150000
50%        -0.070000
75%         0.010000
max         0.580000
Name: ChangeIfHome, dtype: float64

In [31]:
# datetime object containing current date and time
from datetime import datetime
#now = datetime.now()
now = pd.to_datetime('today')
 
# dd/mm/YY H:M:S
#dt_string = now.strftime("%Y/%m/%d %H:%M:%S")

#write in the today's date as the date of prediction
#so this can be catalogued and tracked
#with time as well if there are more recent runs in one day
claims_data['DateOfPredictions'] = now

## Save Data to Vertica

In [32]:
#filter data to columns needed for push to vertica
claims_data_toDB = claims_data[['DATA_SOURCE', 'CLIENT', 'MEMBER_ID', 'EPI_ID',  'PAID_THRU_DATE', 'STAY_ADMIT_DATE',  
                       "Readmission_Likelihood", "SNF_Discharge_Likelihood", "pred_Home_RA", "pred_SNF_RA",
                       'ChangeIfHome','zscore','Recommendation','DateOfPredictions']]

In [33]:
claims_data_toDB.describe()

Readmission_Likelihood  SNF_Discharge_Likelihood  pred_Home_RA  \
count             3096.000000               3096.000000   3096.000000   
mean                 0.286408                  0.031219      0.284277   
std                  0.184794                  0.112272      0.184509   
min                  0.000000                  0.000000      0.000000   
25%                  0.150000                  0.000000      0.150000   
50%                  0.270000                  0.000000      0.260000   
75%                  0.410000                  0.020000      0.400000   
max                  0.990000                  1.000000      0.980000   

       pred_SNF_RA  ChangeIfHome  zscore  
count  3096.000000   3096.000000     0.0  
mean      0.356235     -0.071958     NaN  
std       0.168769      0.140829     NaN  
min       0.000000     -0.790000     NaN  
25%       0.240000     -0.150000     NaN  
50%       0.350000     -0.070000     NaN  
75%       0.470000      0.010000     NaN  
max       0.940000      0.580000     NaN

In [34]:
#set schema name in vertica
schema = 'DATALAB_PRD'

#set table name in vertica
table = 'HomeFirst_Model_Outputs_DEV'


In [35]:
%%time
#Vertica connection to extract data from Vertica table
with vertica_python.connect(**vert_str) as connection:
    #create connection
    cur = connection.cursor()
    
    #delete old records from table
    del_sql = 'drop table '+schema +'.' + table
    cur.execute(del_sql)
    print('Dropped ' +table)

Dropped HomeFirst_Model_Outputs_DEV
Wall time: 442 ms


In [36]:
with vertica_python.connect(**vert_str) as connection:
#create connection
    cur = connection.cursor()

    start = time.time()
    #save pandas dataframe as vertica temp table
    temp = pandas_to_vertica(df = claims_data_toDB,
                      cursor = cur,
                      name = table,
                      schema = schema
                     )

    #push vertica temp table to static table in vertica
    temp.to_db(name=schema+'.'+table, relation_type="table")
    elapsed = round((time.time()-start)/60,2)
    print('Completed in ' + str(elapsed) + ' minute(s).')

Completed in 0.67 minute(s).


## Get NB runtime

In [37]:
#get time of end of code
endCodeTime = time.time()

In [38]:
elapseCodeTime = round(endCodeTime - startCodeTime,2)
print(round(elapseCodeTime/60, 2), 'min - total run time')
print(round((elapseCodeTime/60)/60, 2), 'hr - total run time')

0.84 min - total run time
0.01 hr - total run time


In [39]:
secProw = round(elapseCodeTime/len(claims_data), 4)
print('this run took', secProw, 'secs per input table row')

this run took 0.0162 secs per input table row
